### Spark notebook ###

This notebook will only work in a Jupyter session running on `mathmadslinux2p`.

You can start your own Jupyter session on `mathmadslinux2p` and open this notebook in Chrome on the MADS Windows server by

**Steps**

1. Login to the MADS Windows server using https://mathportal.canterbury.ac.nz/.
2. Download or copy this notebook to your home directory.
3. Open powershell and run `ssh mathmadslinux2p`.
4. Run `start_pyspark_notebook` or `/opt/anaconda3/bin/jupyter-notebook --ip 132.181.129.68 --port $((8000 + $((RANDOM % 999))))`.
5. Copy / paste the url provided in the shell window into Chrome on the MADS Windows server.
6. Open the notebook from the Jupyter root directory (which is your home directory).
7. Run `start_spark()` to start a spark session in the notebook.
8. Run `stop_spark()` before closing the notebook or kill your spark application by hand using the link in the Spark UI.

In [4]:
# Run this cell to import pyspark and to define start_spark() and stop_spark()

import findspark

findspark.init()

import getpass
import pandas
import pyspark
import random
import re

from IPython.display import display, HTML
from pyspark import SparkContext
from pyspark.sql import SparkSession


# Functions used below

def username():
    """Get username with any domain information removed.
    """

    return re.sub('@.*', '', getpass.getuser())


def dict_to_html(d):
    """Convert a Python dictionary into a two column table for display.
    """

    html = []

    html.append(f'<table width="100%" style="width:100%; font-family: monospace;">')
    for k, v in d.items():
        html.append(f'<tr><td style="text-align:left;">{k}</td><td>{v}</td></tr>')
    html.append(f'</table>')

    return ''.join(html)


def show_as_html(df, n=20):
    """Leverage existing pandas jupyter integration to show a spark dataframe as html.
    
    Args:
        n (int): number of rows to show (default: 20)
    """

    display(df.limit(n).toPandas())

    
def display_spark():
    """Display the status of the active Spark session if one is currently running.
    """
    
    if 'spark' in globals() and 'sc' in globals():

        name = sc.getConf().get("spark.app.name")
        
        html = [
            f'<p><b>Spark</b></p>',
            f'<p>The spark session is <b><span style="color:green">active</span></b>, look for <code>{name}</code> under the running applications section in the Spark UI.</p>',
            f'<ul>',
            f'<li><a href="http://mathmadslinux2p.canterbury.ac.nz:8080/" target="_blank">Spark UI</a></li>',
            f'<li><a href="{sc.uiWebUrl}" target="_blank">Spark Application UI</a></li>',
            f'</ul>',
            f'<p><b>Config</b></p>',
            dict_to_html(dict(sc.getConf().getAll())),
            f'<p><b>Notes</b></p>',
            f'<ul>',
            f'<li>The spark session <code>spark</code> and spark context <code>sc</code> global variables have been defined by <code>start_spark()</code>.</li>',
            f'<li>Please run <code>stop_spark()</code> before closing the notebook or restarting the kernel or kill <code>{name}</code> by hand using the link in the Spark UI.</li>',
            f'</ul>',
        ]
        display(HTML(''.join(html)))
        
    else:
        
        html = [
            f'<p><b>Spark</b></p>',
            f'<p>The spark session is <b><span style="color:red">stopped</span></b>, confirm that <code>{username() + " (jupyter)"}</code> is under the completed applications section in the Spark UI.</p>',
            f'<ul>',
            f'<li><a href="http://mathmadslinux2p.canterbury.ac.nz:8080/" target="_blank">Spark UI</a></li>',
            f'</ul>',
        ]
        display(HTML(''.join(html)))


# Functions to start and stop spark

def start_spark(executor_instances=2, executor_cores=1, worker_memory=1, master_memory=1):
    """Start a new Spark session and define globals for SparkSession (spark) and SparkContext (sc).
    
    Args:
        executor_instances (int): number of executors (default: 2)
        executor_cores (int): number of cores per executor (default: 1)
        worker_memory (float): worker memory (default: 1)
        master_memory (float): master memory (default: 1)
    """

    global spark
    global sc

    user = username()
    
    cores = executor_instances * executor_cores
    partitions = cores * 4
    port = 4000 + random.randint(1, 999)

    spark = (
        SparkSession.builder
        .master("spark://masternode2:7077")
        .config("spark.driver.extraJavaOptions", f"-Dderby.system.home=/tmp/{user}/spark/")
        .config("spark.dynamicAllocation.enabled", "false")
        .config("spark.executor.instances", str(executor_instances))
        .config("spark.executor.cores", str(executor_cores))
        .config("spark.cores.max", str(cores))
        .config("spark.executor.memory", f"{worker_memory}g")
        .config("spark.driver.memory", f"{master_memory}g")
        .config("spark.driver.maxResultSize", "0")
        .config("spark.sql.shuffle.partitions", str(partitions))
        .config("spark.ui.port", str(port))
        .appName(user + " (jupyter)")
        .getOrCreate()
    )
    sc = SparkContext.getOrCreate()
    
    display_spark()

    
def stop_spark():
    """Stop the active Spark session and delete globals for SparkSession (spark) and SparkContext (sc).
    """

    global spark
    global sc

    if 'spark' in globals() and 'sc' in globals():

        spark.stop()

        del spark
        del sc

    display_spark()


# Make css changes to improve spark output readability

html = [
    '<style>',
    'pre { white-space: pre !important; }',
    'table.dataframe td { white-space: nowrap !important; }',
    'table.dataframe thead th:first-child, table.dataframe tbody th { display: none; }',
    '</style>',
]
display(HTML(''.join(html)))

### Example notebook ###

The code below provides a template for how you would use a notebook to start spark, run some code, and then stop spark.

**Steps**

- Run `start_spark()` to start a spark session in the notebook (only change the default resources when advised to do so for an exercise or assignment)
- Write and run code interactively, creating additional cells as needed.
- Run `stop_spark()` before closing the notebook or kill your spark application by hand using the link in the [Spark UI](http://mathmadslinux2p.canterbury.ac.nz:8080/).

In [5]:
# Run this cell to start a spark session in this notebook

start_spark(executor_instances=2, executor_cores=1, worker_memory=1, master_memory=1)

spark.dynamicAllocation.enabled,false
spark.app.id,app-20240915200455-1135
spark.master,spark://masternode2:7077
spark.executor.id,driver
spark.driver.memory,1g
spark.driver.host,mathmadslinux2p.canterbury.ac.nz
spark.app.startTime,1726387494695
spark.ui.port,4711
spark.driver.extraJavaOptions,-Dderby.system.home=/tmp/dca129/spark/
spark.executor.memory,1g
spark.rdd.compress,True


In [6]:
# Write your imports here or insert cells below

from pyspark.sql import Row, DataFrame, Window, functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import substring, countDistinct,collect_set

In [7]:
##Q1.a extract the country code in stations
stations = spark.read.text('hdfs:///data/ghcnd/ghcnd-stations.txt')
stations = stations.select(
    F.trim(F.substring(F.col("value"), 1, 11)).cast(StringType()).alias("ID"),
    F.trim(F.substring(F.col("value"), 13, 8)).cast(DoubleType()).alias("LATITUDE"),
    F.trim(F.substring(F.col("value"), 22, 9)).cast(DoubleType()).alias("LONGITUDE"),
    F.trim(F.substring(F.col("value"), 32, 6)).cast(DoubleType()).alias("ELEVATION"),
    F.trim(F.substring(F.col("value"), 39, 2)).cast(StringType()).alias("STATE"),
    F.trim(F.substring(F.col("value"), 42, 30)).cast(StringType()).alias("NAME"),
    F.trim(F.substring(F.col("value"), 73, 3)).cast(StringType()).alias("GSN_FLAG"),
    F.trim(F.substring(F.col("value"), 77, 3)).cast(StringType()).alias("HCN_CRN_FLAG"),
    F.trim(F.substring(F.col("value"), 81, 5)).cast(StringType()).alias("WMO_ID"),
).na.replace('', None)

stations = stations.withColumn("COUNTRY_CODE", substring("ID",1,2))
stations.printSchema()
show_as_html(stations)

root
 |-- ID: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- ELEVATION: double (nullable = true)
 |-- STATE: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- GSN_FLAG: string (nullable = true)
 |-- HCN_CRN_FLAG: string (nullable = true)
 |-- WMO_ID: string (nullable = true)
 |-- COUNTRY_CODE: string (nullable = true)



,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,GSN_FLAG,HCN_CRN_FLAG,WMO_ID,COUNTRY_CODE
0,ACW00011604,17.1167,-61.7833,10.1,None,ST JOHNS COOLIDGE FLD,None,None,None,AC
1,ACW00011647,17.1333,-61.7833,19.2,None,ST JOHNS,None,None,None,AC
2,AE000041196,25.3330,55.5170,34.0,None,SHARJAH INTER. AIRP,GSN,None,41196,AE
3,AEM00041194,25.2550,55.3640,10.4,None,DUBAI INTL,None,None,41194,AE
4,AEM00041217,24.4330,54.6510,26.8,None,ABU DHABI INTL,None,None,41217,AE
5,AEM00041218,24.2620,55.6090,264.9,None,AL AIN INTL,None,None,41218,AE
6,AF000040930,35.3170,69.0170,3366.0,None,NORTH-SALANG,GSN,None,40930,AF
7,AFM00040938,34.2100,62.2280,977.2,None,HERAT,None,None,40938,AF
8,AFM00040948,34.5660,69.2120,1791.3,None,KABUL INTL,None,None,40948,AF
9,AFM00040990,31.5000,65.8500,1010.0,None,KANDAHAR AIRPORT,None,None,40990,AF


In [8]:
##Q1.b left join stations with countries
countries = spark.read.text('hdfs:///data/ghcnd/ghcnd-countries.txt')

countries = countries.select(
    F.trim(F.substring(F.col("value"), 1, 2)).cast(StringType()).alias("CODE"),
    F.trim(F.substring(F.col("value"), 4, 61)).cast(StringType()).alias("NAME")
).na.replace('', None)

station_countries = stations.join(
    countries
    .select(
        'CODE',
        F.col('NAME').alias('COUNTRY_NAME')
        ),
    stations.COUNTRY_CODE == countries.CODE, 
    how = 'left'
).drop('CODE')

# 

station_countries.cache()
show_as_html(station_countries)

,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,GSN_FLAG,HCN_CRN_FLAG,WMO_ID,COUNTRY_CODE,COUNTRY_NAME
0,ACW00011604,17.1167,-61.7833,10.1,None,ST JOHNS COOLIDGE FLD,None,None,None,AC,Antigua and Barbuda
1,ACW00011647,17.1333,-61.7833,19.2,None,ST JOHNS,None,None,None,AC,Antigua and Barbuda
2,AE000041196,25.3330,55.5170,34.0,None,SHARJAH INTER. AIRP,GSN,None,41196,AE,United Arab Emirates
3,AEM00041194,25.2550,55.3640,10.4,None,DUBAI INTL,None,None,41194,AE,United Arab Emirates
4,AEM00041217,24.4330,54.6510,26.8,None,ABU DHABI INTL,None,None,41217,AE,United Arab Emirates
5,AEM00041218,24.2620,55.6090,264.9,None,AL AIN INTL,None,None,41218,AE,United Arab Emirates
6,AF000040930,35.3170,69.0170,3366.0,None,NORTH-SALANG,GSN,None,40930,AF,Afghanistan
7,AFM00040938,34.2100,62.2280,977.2,None,HERAT,None,None,40938,AF,Afghanistan
8,AFM00040948,34.5660,69.2120,1791.3,None,KABUL INTL,None,None,40948,AF,Afghanistan
9,AFM00040990,31.5000,65.8500,1010.0,None,KANDAHAR AIRPORT,None,None,40990,AF,Afghanistan


In [9]:
##Q1.c left join stations with states and the states codes are only provided for statins in the US
states = spark.read.text('hdfs:///data/ghcnd/ghcnd-states.txt')

states = states.select(
    F.trim(F.substring(F.col("value"), 1, 2)).cast(StringType()).alias("CODE"),
    F.trim(F.substring(F.col("value"), 4, 47)).cast(StringType()).alias("NAME")
).na.replace('', None)

stations_states_US = (
    station_countries
    .where(
        F.col('COUNTRY_CODE') == 'US'
    )
    .join(
        states.
        select(F.col('CODE'),
               F.col('NAME').alias('STATE_NAME')
     ),
      station_countries.COUNTRY_CODE == states.CODE,
      how = 'left'
     )
     .drop('CODE')
)
show_as_html(stations_states_US)


,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,GSN_FLAG,HCN_CRN_FLAG,WMO_ID,COUNTRY_CODE,COUNTRY_NAME,STATE_NAME
0,US009052008,43.7333,-96.6333,482.0,SD,SIOUX FALLS (ENVIRON. CANADA),None,None,None,US,United States,None
1,US10RMHS145,40.5268,-105.1113,1569.1,CO,RMHS 1.6 SSW,None,None,None,US,United States,None
2,US10adam001,40.5680,-98.5069,598.0,NE,JUNIATA 1.5 S,None,None,None,US,United States,None
3,US10adam002,40.5093,-98.5493,601.1,NE,JUNIATA 6.0 SSW,None,None,None,US,United States,None
4,US10adam003,40.4663,-98.6537,615.1,NE,HOLSTEIN 0.1 NW,None,None,None,US,United States,None
5,US10adam004,40.4798,-98.4026,570.0,NE,AYR 3.5 NE,None,None,None,US,United States,None
6,US10adam006,40.4372,-98.5912,601.1,NE,ROSELAND 2.8 SW,None,None,None,US,United States,None
7,US10adam007,40.5389,-98.4713,588.9,NE,HASTINGS 5.4 WSW,None,None,None,US,United States,None
8,US10adam008,40.4953,-98.2973,566.9,NE,GLENVIL 2.3 WSW,None,None,None,US,United States,None
9,US10adam010,40.5532,-98.6297,622.1,NE,JUNIATA 6.9 WSW,None,None,None,US,United States,None


In [10]:
## Q1.d first and last year that each station was active
inventory = spark.read.text('hdfs:///data/ghcnd/ghcnd-inventory.txt')
inventory = inventory.select(
    F.trim(F.substring(F.col("value"), 1, 11)).cast(StringType()).alias("ID"),
    F.trim(F.substring(F.col("value"), 13, 8)).cast(DoubleType()).alias("LATITUDE"),
    F.trim(F.substring(F.col("value"), 22, 9)).cast(DoubleType()).alias("LONGITUDE"),
    F.trim(F.substring(F.col("value"), 32, 4)).cast(StringType()).alias("ELEMENT"),
    F.trim(F.substring(F.col("value"), 37, 4)).cast(StringType()).alias("FIRSTYEAR"),
    F.trim(F.substring(F.col("value"), 42, 4)).cast(StringType()).alias("LASTYEAR")
).na.replace('', None)

years_stats = (
    inventory
    .groupBy("ID")
    .agg(
        F.min(F.col("firstyear")).cast(IntegerType()).alias("MIN_FIRSTYEAR"),
        F.max(F.col("lastyear")).cast(IntegerType()).alias("MAX_LASTYEAR")
        )
)
show_as_html(years_stats)

,ID,MIN_FIRSTYEAR,MAX_LASTYEAR
0,AEM00041217,1983,2024
1,AGE00147708,1879,2024
2,AGE00147710,1909,2009
3,AGE00147714,1896,1938
4,AGE00147719,1888,2024
5,AGM00060360,1945,2024
6,AGM00060445,1957,2024
7,AGM00060452,1985,2024
8,AGM00060511,1983,2024
9,AGM00060540,1981,2022


In [11]:
## the number of unique elements each station has collected

elements_stats = (
    inventory
    .groupBy("ID")
    .agg(F.countDistinct(F.col("ELEMENT")).alias("ELEMENT_COUNT"))
    .orderBy("ELEMENT_COUNT", ascending=False)
)

show_as_html(elements_stats)

,ID,ELEMENT_COUNT
0,USW00014607,70
1,USW00013880,70
2,USW00023066,67
3,USW00013958,66
4,USW00093817,65
5,USW00024121,65
6,USW00093058,64
7,USW00014944,64
8,USW00024127,63
9,USW00024156,63


In [12]:
## number of core elements and the number of "other" elements that each station has collected
core_elements = ["PRCP","SNOW","SNWD","TMAX","TMIN"]
core_elements_count =(
    inventory
    .filter(F.col("ELEMENT").isin(core_elements))
    .groupBy("ID")
    .agg(F.count(F.col("ELEMENT")).alias("CORE_ELEMENTS_COUNT")
))
show_as_html(core_elements_count)

,ID,CORE_ELEMENTS_COUNT
0,AEM00041217,3
1,AGE00147708,4
2,AGE00147710,3
3,AGE00147714,3
4,AGE00147719,3
5,AGM00060360,3
6,AGM00060445,4
7,AGM00060452,3
8,AGM00060511,4
9,AGM00060540,4


In [13]:
## number of core elements and the number of "other" elements that each station has collected
core_elements = ["PRCP","SNOW","SNWD","TMAX","TMIN"]
other_elements_count =(
    inventory
    .filter(~F.col("ELEMENT").isin(core_elements))
    .groupBy("ID")
    .agg(F.count(F.col("ELEMENT")).alias("OTHER_ELEMENTS_COUNT")
))
show_as_html(other_elements_count)

,ID,OTHER_ELEMENTS_COUNT
0,AEM00041217,1
1,AGE00147708,1
2,AGE00147710,1
3,AGE00147719,1
4,AGM00060360,1
5,AGM00060445,1
6,AGM00060452,1
7,AGM00060511,1
8,AGM00060540,1
9,AJ000037749,1


In [14]:
## the number of stations collecting all five core elements
core_elements = ["PRCP","SNOW","SNWD","TMAX","TMIN"]

filtered_inventory = inventory.filter(F.col("ELEMENT").isin(core_elements))

stations_with_elements =( 
    filtered_inventory
    .groupBy("ID")
    .agg(collect_set(F.col("ELEMENT")).alias("ELEMENTS_COLLECTED")
))
show_as_html(stations_with_elements)

,ID,ELEMENTS_COLLECTED
0,AEM00041217,"[TMAX, TMIN, PRCP]"
1,AGE00147708,"[TMAX, TMIN, PRCP, SNWD]"
2,AGE00147710,"[TMAX, TMIN, PRCP]"
3,AGE00147714,"[TMAX, TMIN, PRCP]"
4,AGE00147719,"[TMAX, TMIN, PRCP]"
5,AGM00060360,"[TMAX, TMIN, PRCP]"
6,AGM00060445,"[TMAX, TMIN, PRCP, SNWD]"
7,AGM00060452,"[TMAX, TMIN, PRCP]"
8,AGM00060511,"[TMAX, TMIN, PRCP, SNWD]"
9,AGM00060540,"[TMAX, TMIN, PRCP, SNWD]"


In [15]:
stations_with_all_core_elements = (
    stations_with_elements
    .filter(
    F.size(F.col("ELEMENTS_COLLECTED")) == len(core_elements)
))
stations_with_all_core_elements.count()

20482

In [16]:
## the number of stations collecting only precipitation

stations_with_precipitation_only = (
    stations_with_elements
     .filter( (F.size(F.col("ELEMENTS_COLLECTED")) == 1) &
            (F.array_contains(F.col("ELEMENTS_COLLECTED"), "PRCP"))
))
stations_with_precipitation_only.count()

29667

In [17]:
# Run this cell before closing the notebook or kill your spark application by hand using the link in the Spark UI

stop_spark()